In [58]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import transformers
from torch.optim import Adam
from tqdm import tqdm

In [59]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

In [60]:
from utils import FinQA_Dataset

In [61]:
TOKENIZER = T5Tokenizer.from_pretrained("t5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("../finqa_finetune_t5.pth/")
BATCH_SIZE = 4
TESTING = True

/Users/ashish1610dhiman/opt/anaconda3/envs/dlt_project/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [62]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

### Read Test and do inference

In [63]:
finqa_test = torch.load('../finqa_dataset/finqa_test.pth')
len(finqa_test)

1108

In [64]:
if TESTING:
    test_loader = DataLoader(Subset(finqa_test, range(20)), batch_size=2)
else:
    test_loader = DataLoader(finqa_test, batch_size=BATCH_SIZE)

In [65]:
TOKENIZER.decode([x for x in finqa_test[0]['labels'] if x!= -100], skip_special_tokens=True)

'Step 1: Subtract 5735 from 5829. This gives the result: 94'

In [66]:
MODEL.eval();
generated_texts = []
for batch in tqdm(test_loader, desc="Generation batches"):
    input_ids = batch["input_ids"].to(DEVICE)
    attention_mask = batch["attention_mask"].to(DEVICE)
    decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)    
    with torch.no_grad():
        # Use the `generate` method for text generation
        generated_output = MODEL.generate(
            input_ids=input_ids,
            max_length=128,  # Set the desired maximum length for generated text
            num_beams=5,    # You can adjust the number of beams for beam search
            temperature=0.8  # You can adjust the temperature for sampling,
            )
        for output_i in generated_output:
            generated_text = TOKENIZER.decode([x for x in output_i if x!= -100], skip_special_tokens=True)
            generated_texts.append(generated_text)

Generation batches:   0%|          | 0/10 [00:00<?, ?it/s]

Generation batches: 100%|██████████| 10/10 [01:48<00:00, 10.84s/it]


In [67]:
len(generated_texts)

20

In [68]:
import pickle
file_path = 'generated_sentences_test.pkl'
# Open the file in binary read mode and use pickle.load() to load the list
with open(file_path, 'wb') as file:
    pickle.dump(generated_texts, file)